In [45]:
import pandas as pd
import os

from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer

import nltk
from nltk.corpus import stopwords
from nltk.stem import *
from sklearn.linear_model import LogisticRegression

In [46]:
path = "X:/ProgramFiles/JetBrains/PycharmProjects/NBA-predictor/resources/spam-filtering"

filenames = os.listdir(path)
videos = [pd.read_csv(path + '/' + filename, sep=',') for filename in filenames]

dataset = pd.concat(videos)

In [47]:
dataset.sample(frac=1).head(10)

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
158,z12tyfpzkznlghgqz04cjpsp3ymuil05t3g,sheerwill,2015-05-18T13:01:00.402000,I like this song<br />﻿,0
366,z13hxfdrqom5clnaa04cgx5hkpqjexhh3wk0k,DERRICK HANFORD,2014-12-05T22:52:29.316000,White people are going extinct for more inform...,1
188,_2viQ_Qnc6_kxEjV8Ut57SeEGdqSKmC8VGP1K38Q2qY,Scott Johnson,2013-10-18T19:45:43,You guys should check out this EXTRAORDINARY w...,1
312,z125tj4zjlngxfvxr04cg5wyhkmxhxloesk0k,Anders Lundgren,2014-11-07T18:08:44,"#nowplaying ""Weekendmix"" track 04 : Katy Perry...",1
320,z124zbcigmbfvhpru04cepdx2vikibhp42c,Yabbamita DBH,2014-11-12T09:30:20,If the shitty Chinese Government didn't block ...,0
394,z13lsx44zmmqzz2mv22tefeb2undfvcov04,Whami inc.,2014-09-27T17:41:15.536000,go check out our video﻿,1
72,z124fn5ahqnfdbxtg23ihlijyqjqtr1lk,Oh 1080s,2014-11-02T01:08:10,Sub my channel!﻿,1
208,z12lsp3pnmqlirwwk04cepijkvrustgbcgs,Isuzu,2014-11-07T16:38:47,P E A C E &amp; L O V E ! !﻿,0
301,z12vu5grlujzfr4cx23iu5zpdpugzxl4t04,Serkan Kaya,2014-11-08T15:27:27,http://hackfbaccountlive.com/?ref=4436607 psy...,1
13,z121zxaxsq25z5k5o04ch1o5jqqfij3gtm40k,TheUploadaddict,2013-11-28T18:12:12,subscribe like comment﻿,1


In [48]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Mateusz\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [49]:
stop_words = stopwords.words("english")
vectorizer = CountVectorizer(stop_words=stop_words, max_features=4328)
Y = dataset["CLASS"]
wnl = WordNetLemmatizer()

In [50]:
X_lem = vectorizer.fit_transform(dataset["CONTENT"].apply(wnl.lemmatize))

In [51]:
model = svm.SVC(kernel='poly', C=1, degree=2)

In [52]:
from sklearn.model_selection import train_test_split
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_lem, Y, test_size=0.2, random_state=42)

In [53]:
model.fit(X_train1, y_train1)

SVC(C=1, degree=2, kernel='poly')

In [54]:
from sklearn.metrics import *
 
accuracy_score(y_test1, model.predict(X_test1)), precision_score(y_test1, model.predict(X_test1))

(0.7882653061224489, 1.0)

In [55]:
path_to_posts = "X:/ProgramFiles/JetBrains/PycharmProjects/NBA-predictor/resources/comments"

filenames = os.listdir(path_to_posts)
comments = [pd.read_csv(path_to_posts + '/' + filename, sep='\t') for filename in filenames]

comments = pd.concat(comments)

In [56]:
comments.head(10)

,TEAM_ABBREVIATION,COMMENT
0,PHX,This is Grayson Allen’s team. Kd and Book are ...
1,PHX,Donjoe Lampe KD only wins when he is with star...
2,PHX,Grayson carried for 3quarters and KD brought i...
3,NaN,Michelle Witman no
4,PHX,Grayson Allen's shoulders must hurt for carryi...
5,PHX,Grayson better get a spot in the 3P contest. 🔥
6,PHX,3rd straight wins for the #suns!!! what a run ...
7,PHX,It’s time to start considering Durant a greate...
8,PHX,"Well done, KD!"
9,NaN,Chris Paynter hello


In [57]:
comments["SPAM"] = comments["TEAM_ABBREVIATION"].apply(lambda x: 1 if x == "SPAM" else 0)

In [58]:
comments[comments['SPAM'] == 0].describe()

,SPAM
count,24762.0
mean,0.0
std,0.0
min,0.0
25%,0.0
50%,0.0
75%,0.0
max,0.0


In [59]:
test_comments = comments.sample(frac=0.5, random_state=1)

In [60]:
test_comments[test_comments["SPAM"] == 1].describe()

,SPAM
count,1109.0
mean,1.0
std,0.0
min,1.0
25%,1.0
50%,1.0
75%,1.0
max,1.0


In [61]:
X_comments_lem = vectorizer.fit_transform(test_comments["COMMENT"].apply(wnl.lemmatize))
Y_comments = test_comments["SPAM"]

In [62]:
from sklearn.metrics import *

y_pred = model.predict(X_comments_lem)

In [63]:
accuracy_score(Y_comments, y_pred), precision_score(Y_comments, y_pred)

(0.9172798216276478, 0.0)

In [64]:
test_comments[test_comments["SPAM"] == y_pred].describe()


,SPAM
count,12342.0
mean,0.0
std,0.0
min,0.0
25%,0.0
50%,0.0
75%,0.0
max,0.0


In [65]:
from sklearn.model_selection import train_test_split

vectorizer2= CountVectorizer(stop_words=stop_words)

X_lem2 = vectorizer2.fit_transform(comments["COMMENT"].apply(wnl.lemmatize))
Y2 = comments["SPAM"]

X_train, X_test, y_train, y_test = train_test_split(X_lem2, Y2, test_size=0.33, random_state=42)



Y_comments = test_comments["SPAM"]

In [66]:
from sklearn.model_selection import KFold, GridSearchCV


def hypertune_svm(x, y):
    
    lr = LogisticRegression()
    penalty1 = ['l1']
    solvers1 = ['saga', 'liblinear']
    c_values = [100,50, 10, 1.0, 0.1, 0.01]
    lgr_grid1 = dict(solver=solvers1, penalty=penalty1, C=c_values)
    
    penalty2 = ['l2']
    solvers2 = ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga']
    lgr_grid2 = dict(solver=solvers2, penalty=penalty2, C=c_values)
    
    
    
    # model = svm.SVC()
    # kernel = ['linear', 'poly', 'rbf', 'sigmoid']
    # C = [1000, 100, 50, 10, 1.0, 0.1, 0.01]
    # gamma = ['scale']
    # degree = range(1,11)
    # # define grid search
    # grid = dict(kernel=kernel,C=C,gamma=gamma, degree=degree)

    cv = KFold(n_splits=10)
    grid_search = GridSearchCV(estimator=lr, param_grid=lgr_grid1, n_jobs=-1, cv=cv, scoring='precision', error_score=0, verbose=2)
    grid_result = grid_search.fit(x, y)
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))

In [67]:
hypertune_svm(X_train, y_train)

Fitting 10 folds for each of 12 candidates, totalling 120 fits



KeyboardInterrupt



In [ ]:
model2= svm.SVC(kernel='poly', C=1, degree=2)

# model2.fit(X_train, y_train)
# y_pred = model2.predict(X_test)

In [ ]:
# accuracy_score(y_test, y_pred), precision_score(y_test, y_pred)